In [24]:
import boto3
import json
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def get_embedding(prompt_data, boto3_bedrock):
    body = json.dumps({"inputText": prompt_data})
    modelId = "amazon.titan-embed-text-v1"  # (Change this to try different embedding models)
    accept = "application/json"
    contentType = "application/json"

    response = boto3_bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    embedding = response_body.get("embedding")
    return embedding


def invoke_model(prompt_data, question, boto3_bedrock):

    model_input_dict = {'anthropic_version': 'bedrock-2023-05-31',
                        'max_tokens': 20000,
                        'system': prompt_data,
                        "messages" : question}

    body = json.dumps(model_input_dict)

    modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'  # change this to use a different version from the model provider
    accept = "application/json"
    contentType = "application/json"

    response = boto3_bedrock.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    response_string = response_body['content'][0]['text']
    response_string = response_string.strip()

    return response_string


def answer_question(question, boto3_bedrock, prompt=""):
    user_request = "<request>" + question + "</request>"
    messages = [{"role": "user", "content": user_request}]
    return invoke_model(prompt, messages, boto3_bedrock)


def compute_closest_texts(question, embeddings, boto3_bedrock, embedding_column = 'embedding'):
    target_embedding = get_embedding(question, boto3_bedrock)

    vector_array = np.array(target_embedding)
    matrix_array = np.array([emb for emb in embeddings[embedding_column]])

    vector_array = vector_array.reshape(1, -1)

    # Compute cosine similarity
    similarities = cosine_similarity(vector_array, matrix_array)

    n_texts = len(similarities)-1
    
    # Flatten the similarities matrix and get the indices of the top 3 similarities
    top_indices = np.argpartition(similarities.flatten(), -n_texts)[-n_texts:]
    top_indices = top_indices[np.argsort(similarities.flatten()[top_indices])][::-1]
    
    return top_indices


def enhance_prompt(question, df, boto3_bedrock, num_contexts=5):
    top_indices=compute_closest_texts(question, df, boto3_bedrock, embedding_column = 'embedding')

    prompt = f"""
    <task_instructions> Eres un experto en política colombiana. El usuario te va a hacer una pregunta
    y tú debes responderla basado en los textos de contexto que están a continuación. Evita
    decir cosas que no se mencionen en los textos. Si los textos no contienen información relevante para
    responder la pregunta, responde que no tienes una respuesta. En tu respuesta evita mencionar los textos,
    con frases como "de acuerdo con los textos proporcionados". Tu respuesta debe ser escrita en un párrafo y evita
    escribirla como una lista de puntos.
    """

    for i in range(num_contexts):
        prompt += f""" 
        <context_text>
        {df.loc[top_indices[i], 'content']}
        </context_text>
        """

    return prompt

Inicializar el cliente a Bedrock:

In [2]:
boto3_bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name="us-east-1"
)

In [3]:
model_input_dict = {'anthropic_version': 'bedrock-2023-05-31',
                    'max_tokens': 20000,
                    'system': '',
                    "messages" : [{"role": "user", "content": "Cuál es la capital de Colombia?"}]}

body = json.dumps(model_input_dict)

modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

response = boto3_bedrock.invoke_model(
    body=body, modelId=modelId, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())

response_string = response_body['content'][0]['text']
response_string = response_string.strip()

print(response_string)

La capital de Colombia es Bogotá.

Bogotá es la ciudad más grande y poblada de Colombia. Está ubicada en la región central del país, en la cordillera Oriental de los Andes. Además de ser la capital política, Bogotá es el principal centro económico, cultural y turístico de Colombia.

Algunas características importantes de Bogotá:

- Fue fundada el 6 de agosto de 1538 por Gonzalo Jiménez de Quesada.

- Tiene una población metropolitana de aproximadamente 8 millones de habitantes.

- Su altitud promedio es de 2.625 metros sobre el nivel del mar.

- Alberga la sede del gobierno colombiano con la Casa de Nariño (residencia del Presidente) y los principales ministerios.

- Es un importante centro financiero, empresarial y de servicios de América Latina.

- Posee numerosos museos, teatros, centros culturales y sitios turísticos como La Candelaria, el Cerro de Monserrate y Usaquén.

Así que en resumen, Bogotá es la indiscutible capital política, económica y cultural de la República de Colombia

### Ejemplo sin prompt:

In [4]:
question='Cuál es el escándalo por los carrotanques?'

print(answer_question(question, boto3_bedrock))

El escándalo de los "carrotanques" o "narcosobrinos" se refiere a un caso de narcotráfico ocurrido en 2015 en Venezuela e involucró a dos sobrinos de la ahora ex primera dama venezolana, Cilia Flores. Los hechos principales fueron:

- Efraín Antonio Campo Flores y Franqui Francisco Flores de Freitas, sobrinos de Cilia Flores y por tanto familiares cercanos al expresidente Nicolás Maduro, fueron detenidos en Haití en 2015 cuando intentaban hacer una entrega de 800 kilos de cocaína a supuestos miembros del cartel de Sinaloa en México.

- Fueron acusados en Estados Unidos de haber utilizado identidades diplomáticas falsas y facturar la droga como "carga de asistencia humanitaria" en Venezuela para sacarla del país.

- El juicio demostró que los narcosobrinos buscaban hacer negocios con el cartel mexicano usando sus contactos en el gobierno venezolano.

- En 2017 fueron declarados culpables en una corte de Nueva York y condenados a 18 años de prisión por narcotráfico.

- El caso generó un 

### Retrieval Augmented Generation (RAG):

In [6]:
df = pd.read_csv("noticias_politica_2024.csv")

In [7]:
df['embedding'] = ''
for i in range(len(df)):
    df.at[i, 'embedding'] = get_embedding(df.loc[i, 'content'], boto3_bedrock)

In [8]:
df

content  \
0    En el expediente contra la campaña Petro Presi...   
1    No paran los ataques del presidente de la Repú...   
2    SEMANA: Usted hizo parte del Consejo Superior ...   
3    No guardó silencio el presidente de la Repúbli...   
4    Este domingo 12 de mayo, el alcalde de Bogotá,...   
..                                                 ...   
835  De manera sorpresiva, el presidente de la Repú...   
836  El alcalde de Bogotá, Carlos Fernando Galán, c...   
837  El expresidente Álvaro Uribe se refirió a la d...   
838  El sistema de salud está en crisis. Esa es una...   
839  El expresidente Álvaro Uribe Vélez rompe el si...   

                         date  \
0    2024-05-13T00:30:32.386Z   
1    2024-05-12T23:12:48.839Z   
2    2024-05-13T15:03:00.019Z   
3    2024-05-12T20:07:33.567Z   
4    2024-05-12T18:29:39.661Z   
..                        ...   
835  2024-04-10T16:11:45.297Z   
836  2024-04-10T15:32:24.729Z   
837  2024-04-10T15:28:41.947Z   
838  2024-04-10T15:17:32.108Z   
839  2024-04-10T14:50:41.950Z   

                                              headline  \
0    Expediente contra Petro: CNE afirma que se omi...   
1    Petro se pegó de Bernie Sanders para atacar a ...   
2    “Recomiendo a Petro que vaya al psicólogo porq...   
3    Petro responde con risa a acusaciones de Anton...   
4    Alcalde Carlos Fernando Galán anuncia su prime...   
..                                                 ...   
835  Presidente Petro reveló que en Caracas sostuvo...   
836  Atención: alcalde Carlos Fernando Galán anunci...   
837  Álvaro Uribe asegura que su llamado a juicio e...   
838  Grave alerta de las EPS por “complejo” panoram...   
839  Las 10 frases más reveladoras de la respuesta ...   

                                           description  \
0    Estos son los hallazgos sobre el uso de una ae...   
1    El jefe de Estado en una publicación en su cue...   
2    El reconocido psicólogo Humberto Rosanía, quie...   
3    El jefe de Estado se pronunció en sus redes so...   
4    El mandatario de Bogotá busca financiación par...   
..                                                 ...   
835  El martes (9 de abril), el jefe de Estado tuvo...   
836  El mandatario aseguró que hubo operativos en t...   
837  El exmandatario se refirió a la decisión judic...   
838  Ana María Vesga, presidenta de Acemi, el gremi...   
839  El exmandatario colombiano se pronunció sobre ...   

                                             embedding  
0    [0.38358792662620544, -0.3096645772457123, 0.0...  
1    [0.408599317073822, -0.10449740290641785, 0.21...  
2    [0.30658578872680664, -0.319197416305542, 0.28...  
3    [0.32899171113967896, -0.30707383155822754, 0....  
4    [0.20697084069252014, -0.497946172952652, 0.02...  
..                                                 ...  
835  [0.6004253625869751, -0.428305983543396, 0.350...  
836  [0.7651423215866089, -0.14719359576702118, -0....  
837  [0.5359693169593811, -0.2784307897090912, 0.16...  
838  [0.35191458463668823, -0.2653442621231079, 0.2...  
839  [0.623603880405426, -0.22473417222499847, 0.26...  

[840 rows x 5 columns]

In [29]:
df['embedding']

0      [0.38358792662620544, -0.3096645772457123, 0.0...
1      [0.408599317073822, -0.10449740290641785, 0.21...
2      [0.30658578872680664, -0.319197416305542, 0.28...
3      [0.32899171113967896, -0.30707383155822754, 0....
4      [0.20697084069252014, -0.497946172952652, 0.02...
                             ...                        
835    [0.6004253625869751, -0.428305983543396, 0.350...
836    [0.7651423215866089, -0.14719359576702118, -0....
837    [0.5359693169593811, -0.2784307897090912, 0.16...
838    [0.35191458463668823, -0.2653442621231079, 0.2...
839    [0.623603880405426, -0.22473417222499847, 0.26...
Name: embedding, Length: 840, dtype: object

In [9]:
df.loc[0, 'content']

'En el expediente contra la campaña Petro Presidente de 2022 reposan pruebas sobre la presunta omisión del reporte de las horas de vuelo que se contrataron con la compañía Sociedad Aérea de Ibagué S.A.S., conocida también como Sadi.La cuestión de la avioneta que llevó a Gustavo Petro en algunas de sus correrías por todo el país venía siendo objeto de debate de la opinión pública desde hace varios meses, pero ahora el Consejo Nacional Electoral (CNE) lo tuvo en cuenta para abrir la investigación contra el actual mandatario.“Se observa la presunta manipulación de la información en relación con la generación de pagos por servicios de transporte aéreo realizados por la empresa Sadi toda vez que, de acuerdo a la información allegada por la Dian, se logra evidenciar la generación sistemática de facturas electrónicas que posteriormente son objeto de anulación, generando en consecuencia, una nueva facturación que modifica el valor y el concepto inicial”, se lee en el documento presentado por l

In [37]:
question='Cuál es el escándalo por los carrotanques?'

In [38]:
top_indices=compute_closest_texts(question, df, boto3_bedrock, embedding_column = 'embedding')

In [39]:
df.loc[top_indices[0], 'content']

'Un escándalo de corrupción estalló en Colombia el 3 de mayo, luego de que Sneyder Pinilla, quien fuera subdirector de manejo del riesgo en la Unidad Nacional de Gestión del Riesgo de Desastres (UNGRD), testificara sobre las investigaciones que se adelantan en el caso de los carrotanque para la Guajira. Mientras avanzan las indagaciones, salió a relucir un video en el que el hoy presidente Gustavo Petro, promocionaba la candidatura de Olmedo López, exdirector de la UNGRD, que salió salpicado en el escándalo.En el video promocional, el hoy mandatario de los colombianos recomendaba a los ciudadanos que votaran “por gente decente”, en alusión al partido por el cual se lanzó al Senado Olmedo López. “Este 11 de marzo, no vote por los corruptos al Congreso de la República”, decía Petro en la pieza audiovisual, en invitaba a depositar el voto “en Senado y Cámara, por Decentes”.‘Nace para quitarle $50 billones a la corrupción’Luego del espaldarazo que le daba Petro al partido Decentes, aparecí

In [40]:
df.loc[top_indices[1], 'content']

'El director de la Unidad Nacional de Gestión del Riesgo, Carlos Carrillo, reconoció una vez más que la entidad que dirige fue cooptada por la corrupción. Lo hizo este lunes 6 de mayo desde la localidad de Ciudad Bolívar, en Bogotá.“Y no en este gobierno, esto viene desde que se creó la entidad. Esa clase política no se fue el 7 de agosto de 2022, a esa clase política corrupta todavía nos sigue correspondiendo sacarla del Gobierno. Sacarla“, dijo. El público que asistía al auditorio aplaudió.Carrillo manifestó: “Se colaron unos y traicionaron al señor presidente Gustavo Petro y traicionaron el proyecto de cambio. Esas personas no caben en este gobierno”.Los corruptos, añadió, “que se vayan a la derecha, al centro, pero en la izquierda no tienen espacio. No lo pueden tener. Eso es lo que estamos haciendo. Yo quisiera ver que otro gobierno hubiera sido capaz de coger la UNGRD, que es tan poderosa, y abrir la caja de pandora. Ninguno”.Carrillo intenta desmarcar al gobierno de Gustavo Petr

In [41]:
df.loc[top_indices[2], 'content']

'Luis Fernando Velasco descartó que tenga que ver algo con el escándalo de los carrotanques parqueados en La Guajira. Le respondió al exdirector Olmedo López quien sugirió en una entrevista que el alto funcionario del Gobierno estaría relacionado.“Es el típico caso del raterito que lo cogen con las manos en la masa y trata de generar una confusión para que la gente intente ver hacia otro lado. No ha habido señalamientos porque, si hay señalamientos, yo reaccionaré jurídicamente. Todo son rumores”, detalló el ministro Velasco.Velasco aclaró que no tiene nada que ver con el escándalo. “No firmé un solo contrato o convenio. Atendí la emergencia del Nevado del Ruiz con un convenio que tenía con las fuerzas militares”, dijo.El ministro del Interior llegó acompañado al búnker de la Fiscalía junto a su abogado defensor Ernesto Velasco Chaves, junto a quien presentó una demanda por injuria y calumnia en contra de Silvio Ramírez, un ciudadano que lo relacionó con el escándalo de los carrotanque

In [42]:
top_indices=compute_closest_texts(question, df, boto3_bedrock, embedding_column = 'embedding')

prompt = f"""
<task_instructions> Eres un experto en política colombiana. El usuario te va a hacer una pregunta
y tú debes responderla basado en los textos de contexto que están a continuación. Evita
decir cosas que no se mencionen en los textos. Tu respuesta debe ser escrita en un párrafo y evita
escribirla como una lista de puntos.
"""

num_contexts = 5

for i in range(num_contexts):
    prompt += f""" 
    <context_text>
    {df.loc[top_indices[i], 'content']}
    </context_text>
    """

In [43]:
print(prompt)


<task_instructions> Eres un experto en política colombiana. El usuario te va a hacer una pregunta
y tú debes responderla basado en los textos de contexto que están a continuación. Evita
decir cosas que no se mencionen en los textos. Tu respuesta debe ser escrita en un párrafo y evita
escribirla como una lista de puntos.
 
    <context_text>
    Un escándalo de corrupción estalló en Colombia el 3 de mayo, luego de que Sneyder Pinilla, quien fuera subdirector de manejo del riesgo en la Unidad Nacional de Gestión del Riesgo de Desastres (UNGRD), testificara sobre las investigaciones que se adelantan en el caso de los carrotanque para la Guajira. Mientras avanzan las indagaciones, salió a relucir un video en el que el hoy presidente Gustavo Petro, promocionaba la candidatura de Olmedo López, exdirector de la UNGRD, que salió salpicado en el escándalo.En el video promocional, el hoy mandatario de los colombianos recomendaba a los ciudadanos que votaran “por gente decente”, en alusión al pa

In [44]:
print(answer_question(question, boto3_bedrock, prompt))

El escándalo de los carrotanques se refiere a un caso de presunta corrupción en la Unidad Nacional de Gestión del Riesgo de Desastres (UNGRD) de Colombia. Según las denuncias, se habrían entregado sobornos millonarios a los presidentes del Senado y la Cámara de Representantes con dinero proveniente del contrato para la compra de 40 carrotanques que debían llevar agua a La Guajira, una región afectada por la sequía.

El exsubdirector de la UNGRD, Sneyder Pinilla, reveló que se pagaron 3.000 millones de pesos al presidente del Senado, Iván Name, y 1.000 millones al presidente de la Cámara, Andrés Calle, supuestamente para que aprobaran las reformas impulsadas por el gobierno del presidente Gustavo Petro. Los carrotanques adquiridos a precios inflados nunca funcionaron correctamente en La Guajira y permanecieron parqueados sin prestar el servicio que se requería. Varios organismos de control y la Fiscalía investigan presuntos sobrecostos, irregularidades en la contratación y el posible de

In [25]:
question='Qué ha pasado con Nicolás Petro?'
prompt = enhance_prompt(question, df, boto3_bedrock)

print(answer_question(question, boto3_bedrock, prompt))

Nicolás Petro, hijo del presidente Gustavo Petro, se ha visto involucrado en un escándalo por presuntas irregularidades en el financiamiento de la campaña presidencial de su padre en 2022. Nicolás Petro confesó ante las autoridades que recibió dineros de fuentes cuestionables, como empresarios y exnarcotraficantes, que no fueron reportados oficialmente para la campaña. Algunos de los puntos clave sobre el caso de Nicolás Petro son:

Nicolás confesó haber recibido millonarios aportes en efectivo de personas como el empresario Euclides Torres, el excapo 'Hombre Marlboro' y otros, para financiar eventos y logística de la campaña de Petro. Reconoció haber desviado parte de esos dineros para su propio beneficio. 

Aseguró que su padre, el presidente Petro, estaba al tanto de los aportes de Euclides Torres aunque no de los detalles específicos del manejo de esos recursos.

Nicolás Petro está siendo investigado por la Fiscalía por los presuntos delitos de lavado de activos y enriquecimiento i

In [26]:
question='Hazme un resumen sobre el estado de la reforma a la salud'

prompt = enhance_prompt(question, df, boto3_bedrock)

print(answer_question(question, boto3_bedrock, prompt))

El proceso de la reforma a la salud impulsada por el gobierno de Gustavo Petro ha sido bastante complejo y polémico. Inicialmente se propuso una reforma que buscaba acabar con las EPS y establecer un modelo de salud público completamente estatal. Sin embargo, esta primera propuesta fue muy criticada por diversos actores como pacientes, médicos, academia y fue finalmente archivada en el Congreso.

Después de esto, el gobierno negoció con las EPS una nueva versión de la reforma que mantiene algunas ideas originales pero también incorpora varios cambios. En esta nueva versión se conserva el giro directo de recursos al Estado a través de la ADRES, se transforma a las EPS en "Gestoras de Salud y Vida" sin riesgo financiero, y se les da poder a gobernadores y alcaldes en la administración de hospitales públicos. 

No obstante, esta nueva propuesta también ha generado críticas de senadores que consideran que se mantendrán problemas estructurales, que sigue sin haber suficiente concertación co